# Simulate data

Here we will draw $n=1,000$ from the model:
$$
X_i \sim N(\mu_x, \sigma^2_{x})\\
Y_i|X_i \sim N(\beta_0 + \beta_1 X_i + \beta_2X_i^2, \sigma^2_{y.x} )
$$
With parameters defined below.

In [1]:
import numpy as np
# simulate data
np.random.seed(10)
n = 10000
x = 0.5 + np.random.normal(0,1,n)
y = 2 + x + 3*(x**2) + np.random.normal(0,1,n)

The population average partial derivative is easy to compute analytically, and given by:

$$
\begin{align}
APD = E\left[\frac{\partial E[Y_i|X_i]}{\partial X_i}\right] &= \beta_1 + 2\times \beta_2 \times E[X_i]\\
&= 1 + 2\times (3)\times (0.5)\\
&= 4
\end{align}
$$

# Estimation and Inference

Using the plug-in principle, we can estimate the APD in two ways. First, if we know the analytical formula, like above, we can simply use the sample analogs:

$$
\widehat{APD} = \widehat{\beta}_1 + 2\times \widehat{\beta}_2 \times E_n[X_i]
$$

Where $E_n[X_i] = \frac{1}{n}\sum_{i} X_i$ is the empirical mean. This leads to the following:

In [2]:
import statsmodels.formula.api as smf
data = {"x":x, "y":y}
# fit regression
ols=smf.ols(formula = 'y ~ x+np.power(x, 2)', data = data).fit()

# plug-in estimate of APD using analytical formula 
APD1 = ols.params[1] + 2*ols.params[2]*np.mean(x)
APD1

4.021505239645409

We obtain 4.02, which is pretty close to the truth.

Now, recall the derivative can be defined as :
    
$$
\left. \frac{\partial E[Y_i \mid X_i]}{\partial X_i}\right|_{X_i=x} = \lim_{h\to 0} \frac{E[Y_i \mid X_i= x +h] - E[Y_i \mid X_i = x - h]}{2h}
$$
    
This formula suggests an alternative to approximate the derivative numerically, by computing the above difference with a small enough $h$. 

Thus, appealing again to the plug-in principle, we can estimate the APD using a numerical approximation, and compute:
    
$$
\widehat{\text{APD}} := E_{n}\left[\frac{\widehat{E}[Y_i \mid x_i +h] - \widehat{E}[Y_i \mid x_i - h]}{2h}\right] = \frac{1}{n} \sum_{i}\left[\frac{\widehat{E}[Y_i \mid x_i +h] - \widehat{E}[Y_i \mid x_i - h]}{2h}\right]
$$
This leads to the following estimate, using $h=0.0001$.

In [3]:
# plug-in estimate of APD using numerical derivative
h = 0.0001
yp = ols.predict({"x":x + h})
ym = ols.predict({"x":x - h})
dx = (yp - ym)/(2*h)
APD2 = np.mean(dx)
APD2

4.021505239646939

Which is essentially numerically identical to the previous estimate.

# Inference using the nonparametric bootstrap

One way to perform inference on the APD is using the nonparametric bootstrap. Here we will bootstrap both estimators at the same time. 
Note you need to **bootstrap the whole procedure**. That is:
* Sample with replacement $n$ rows from the data;
* Refit the OLS model;
* Recompute the APD using the refited model and resampled data.

In [4]:
# create bootstrap function
def boot_fun():
    # sample rows
    idx=np.random.choice(np.arange(n), size=n, replace=True)
    
    # fit OLS to boostrapped cases
    x_boot   = x[idx]
    y_boot   = y[idx]
    data = {"x":x_boot, "y":y_boot}
    ols=smf.ols(formula = 'y ~ x+np.power(x, 2)', data = data).fit()

    # recompute plug-in estimate using analytical formula 
    APD1 = ols.params[1] + 2*ols.params[2]*np.mean(x_boot)
    
    # recompute plug-in estimate using numerical derivative 
    h = 0.0001
    yp = ols.predict({"x":x_boot + h})
    ym = ols.predict({"x":x_boot - h})
    dx = (yp - ym)/(2*h)
    APD2 = np.mean(dx)
    APD2
    
    # return both
    return(APD1,APD2)

In [5]:
# now replicate the function 10,000 times
# this gives us 10,000 bootstrap samples
boot_samples=np.zeros((10000,2))
for i in range(10000):
    result=boot_fun()
    boot_samples[i,0]=result[0]
    boot_samples[i,1]=result[1]

Now we can construct confidence intervals. Here we will use two methods: (1) the percentile method; and, (2) the normal approximation method.

1. Percentile method

In [6]:
# set significance level
alpha = 0.05

# Percentile CI

## CI for APD1
print("Percentile CI for APD1:",np.quantile(boot_samples[:,0],[alpha/2,1-alpha/2]))


## CI for APD2
print("Percentile CI for APD2:",np.quantile(boot_samples[:,1], (alpha/2, 1-alpha/2)))

Percentile CI for APD1: [3.90060135 4.13968833]
Percentile CI for APD2: [3.90060135 4.13968833]


2. Normal approximation method

In [7]:
# Normal approximation CI

## critical threshold
from scipy.stats import norm
z = norm.ppf(1-alpha/2)


## compute bootstrap standard errors
sd_boot_1 = np.std(boot_samples[:,0])
sd_boot_2 = np.std(boot_samples[:,1])

## CI for APD1
print("Normal approximation CI for APD1:",APD1-z*sd_boot_1,APD2+z*sd_boot_1)

## CI for APD2
print("Normal approximation CI for APD2:",APD2-z*sd_boot_2,APD2+z*sd_boot_2)

Normal approximation CI for APD1: 3.902465111316791 4.140545367975556
Normal approximation CI for APD2: 3.902465111318246 4.1405453679756326
